In [17]:
import numpy as np
import tensorflow as tf
import pandas as pd

from keras import layers, models, optimizers, applications
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import os

In [24]:
width = 150
height = 150

TRAIN_FOLDER_PATH = "data/train"
TEST_FOLDER_PATH = "data/test1"
TRAIN_FILE_NAMES = os.listdir(TRAIN_FOLDER_PATH)
TEST_FILE_NAMES = os.listdir(TEST_FOLDER_PATH)

x_train_paths = []
y_train = []
for file in TRAIN_FILE_NAMES:
    full_path = os.path.join(TRAIN_FOLDER_PATH, file)
    x_train_paths.append(full_path)
    
    y_train.append(int(file.split(".")[0] == "dog"))

x_test_paths = []
for file in TEST_FILE_NAMES:
    full_path = os.path.join(TEST_FOLDER_PATH, file)
    x_test_paths.append(full_path)


model=applications.VGG16(weights="imagenet", include_top=False, input_shape=(width, height, 3))
# model.summary()

# extract features
def get_features(paths):
    features = []
    for file in paths:
        img = load_img(file, target_size=(width, height))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        feature = model.predict(img)
        features.append(feature)
    features = np.array(features)
    return features.reshape(features.shape[0], 4*4*512)


x_train = get_features(x_train_paths)
x_test = get_features(x_test_paths)
clf=LogisticRegression(solver="lbfgs")
clf.fit(x_train, y_train)
predictions=clf.predict(x_test)
print(predictions)

# predictions = np.argmax(model.predict(test), axis=1)
predictions = pd.DataFrame(data=[[i + 1, predictions[i]] for i in range(len(predictions))],
                           columns=['id', 'label'])

predictions.to_csv('data/guess.csv', index=False)

C:\Users\danie\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[1 0 0 ... 1 0 0]
